In [1]:
import geopandas as gpd
from shapely.geometry import Point

# Load the dataset
from geopandas.datasets import get_path

cities = gpd.read_file(get_path('naturalearth_cities'))


/tmp/ipykernel_39940/464501985.py:7: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_cities' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  cities = gpd.read_file(get_path('naturalearth_cities'))


In [2]:
import pandas as pd

In [3]:
countries = pd.read_csv("data/country_info.csv")
countries

,name,code,capital
0,Albania,al,Tirana
1,Armenia,am,Yerevan
2,Australia,au,Canberra
3,Austria,at,Vienna
4,Azerbaijan,az,Baku
5,Belgium,be,Brussels
6,Croatia,hr,Zagreb
7,Cyprus,cy,Nicosia
8,Czechia,cz,Prague
9,Denmark,dk,København


In [4]:
# Filter for European capitals (manually or with an additional dataset)
european_capitals = countries["capital"]

european_cities = cities[cities['name'].isin(european_capitals)]

# Reproject to a metric CRS (EPSG:3035)
european_cities = european_cities.to_crs(epsg=3035)

# Compute pairwise distances
distances = european_cities.geometry.apply(
    lambda city: european_cities.distance(city)
)

# Convert distances to a DataFrame for readability
import pandas as pd
distance_df = pd.DataFrame(distances.values.tolist(),
                           index=european_cities['name'],
                           columns=european_cities['name'])
distance_df_km = distance_df / 1000
distance_df_km.head()

name,San Marino,Luxembourg,Monaco,Ljubljana,Bratislava,Podgorica,Bern,Skopje,Reykjavík,Rabat,...,Madrid,Stockholm,Amsterdam,Berlin,Athens,Vienna,London,Moscow,Rome,Paris
name,,,,,,,,,,,,,,,,,,,,,
San Marino,0.000000,793.187699,406.385851,286.797948,592.189117,578.338893,511.086762,764.140910,3099.359842,1985.668334,...,1387.303070,1753.433237,1088.861755,957.030918,1152.241692,563.123967,1260.407706,2215.185157,225.783903,948.745271
Luxembourg,793.187699,0.000000,659.545405,741.259918,821.261000,1285.920184,315.676808,1453.332143,2308.397823,2018.610383,...,1276.322674,1324.360680,316.649206,603.339626,1902.743095,765.856559,489.923917,2207.368391,987.925292,287.341398
Monaco,406.385851,659.545405,0.000000,618.087723,898.094760,976.670478,352.579145,1162.612102,2916.383658,1625.680458,...,988.005556,1878.000260,975.011500,1072.217846,1513.276765,852.872041,1031.496168,2520.052810,463.396592,689.639178
Ljubljana,286.797948,741.259918,618.087723,0.000000,305.690766,548.807910,549.804379,712.141160,2991.576154,2238.790467,...,1600.830835,1495.161871,988.020883,723.750448,1169.663977,277.071176,1231.136216,1933.349077,489.407578,967.699977
Bratislava,592.189117,821.261000,898.094760,305.690766,0.000000,652.415653,739.318867,760.751185,2922.625840,2523.704371,...,1860.351135,1245.520598,985.789892,553.540765,1243.935347,56.224988,1291.093344,1631.003637,784.856524,1092.144862


In [5]:
distance_df_km.describe()

name,San Marino,Luxembourg,Monaco,Ljubljana,Bratislava,Podgorica,Bern,Skopje,Reykjavík,Rabat,...,Madrid,Stockholm,Amsterdam,Berlin,Athens,Vienna,London,Moscow,Rome,Paris
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,...,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,1510.729190,1612.618026,1679.636000,1404.922161,1363.154610,1501.051407,1581.812207,1543.266255,3296.539027,2965.644484,...,2380.761255,1842.647737,1702.266574,1486.281943,1840.457586,1366.312297,1907.661801,2095.737747,1608.526968,1776.663922
std,1736.175374,1786.095871,1778.577919,1715.066256,1680.249044,1678.715926,1777.311383,1656.148580,1779.314733,1911.252404,...,1883.864236,1637.417451,1783.044117,1695.173877,1618.939510,1688.040089,1828.109921,1454.022908,1733.075143,1822.286158
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,670.212784,802.612636,898.094760,618.087723,739.318867,621.253755,739.318867,760.751185,2459.901546,2145.827531,...,1477.798222,1229.765007,975.011500,771.012290,1072.148739,684.675850,1231.136216,1568.117780,764.104338,1028.259319
50%,1152.241692,1324.360680,1377.319496,1103.309769,971.601249,1285.920184,1235.300178,1314.267865,2991.576154,2623.173532,...,2035.162232,1547.134580,1414.342149,1072.217846,1655.955050,951.400649,1679.924406,1905.686519,1317.386380,1492.430514
75%,1780.585752,1704.020426,1878.137422,1636.129597,1542.619824,1926.616176,1732.612245,1932.724318,3662.987302,3465.206071,...,2765.793112,2139.417689,1855.412523,1797.463020,2243.943189,1594.165925,2085.276254,2327.636800,1900.460163,1909.838411
max,12263.769177,12542.952472,12656.767026,12044.609045,11801.774379,11765.857224,12539.566033,11601.313142,13413.100915,14224.472280,...,13644.326045,11584.829782,12533.521711,11969.525694,11503.216179,11855.583155,12891.318423,10399.269007,12325.732706,12830.105202


In [6]:
len(european_capitals), len(european_cities)

(53, 49)

In [7]:
set(european_capitals) - set(european_cities["name"])

{'Andorra la Vella', nan}

In [8]:
set(european_cities["name"])

{'Amsterdam',
 'Ankara',
 'Athens',
 'Baku',
 'Belgrade',
 'Berlin',
 'Bern',
 'Bratislava',
 'Brussels',
 'Bucharest',
 'Budapest',
 'Canberra',
 'Chi?in?u',
 'Dublin',
 'Helsinki',
 'Jerusalem',
 'Kyiv',
 'København',
 'Lisbon',
 'Ljubljana',
 'London',
 'Luxembourg',
 'Madrid',
 'Minsk',
 'Monaco',
 'Moscow',
 'Nicosia',
 'Oslo',
 'Paris',
 'Podgorica',
 'Prague',
 'Rabat',
 'Reykjavík',
 'Riga',
 'Rome',
 'San Marino',
 'Sarajevo',
 'Skopje',
 'Sofia',
 'Stockholm',
 'Tallinn',
 'Tbilisi',
 'Tirana',
 'Valletta',
 'Vienna',
 'Vilnius',
 'Warsaw',
 'Yerevan',
 'Zagreb'}

In [10]:
distance_df_km.to_csv("data/distances.csv")
# distance_df_km = pd.read_csv("data/distances.csv")

In [13]:
votes_df = pd.read_csv("data/votes.csv")

In [14]:
distance_df_km.head()

name,San Marino,Luxembourg,Monaco,Ljubljana,Bratislava,Podgorica,Bern,Skopje,Reykjavík,Rabat,...,Madrid,Stockholm,Amsterdam,Berlin,Athens,Vienna,London,Moscow,Rome,Paris
name,,,,,,,,,,,,,,,,,,,,,
San Marino,0.000000,793.187699,406.385851,286.797948,592.189117,578.338893,511.086762,764.140910,3099.359842,1985.668334,...,1387.303070,1753.433237,1088.861755,957.030918,1152.241692,563.123967,1260.407706,2215.185157,225.783903,948.745271
Luxembourg,793.187699,0.000000,659.545405,741.259918,821.261000,1285.920184,315.676808,1453.332143,2308.397823,2018.610383,...,1276.322674,1324.360680,316.649206,603.339626,1902.743095,765.856559,489.923917,2207.368391,987.925292,287.341398
Monaco,406.385851,659.545405,0.000000,618.087723,898.094760,976.670478,352.579145,1162.612102,2916.383658,1625.680458,...,988.005556,1878.000260,975.011500,1072.217846,1513.276765,852.872041,1031.496168,2520.052810,463.396592,689.639178
Ljubljana,286.797948,741.259918,618.087723,0.000000,305.690766,548.807910,549.804379,712.141160,2991.576154,2238.790467,...,1600.830835,1495.161871,988.020883,723.750448,1169.663977,277.071176,1231.136216,1933.349077,489.407578,967.699977
Bratislava,592.189117,821.261000,898.094760,305.690766,0.000000,652.415653,739.318867,760.751185,2922.625840,2523.704371,...,1860.351135,1245.520598,985.789892,553.540765,1243.935347,56.224988,1291.093344,1631.003637,784.856524,1092.144862


In [15]:
distance_stack = distance_df_km.stack().rename("distance").to_frame()
distance_stack.index = distance_stack.index.rename(["from", "to"])
distance_stack = distance_stack.reset_index()
distance_stack

,from,to,distance
0,San Marino,San Marino,0.000000
1,San Marino,Luxembourg,793.187699
2,San Marino,Monaco,406.385851
3,San Marino,Ljubljana,286.797948
4,San Marino,Bratislava,592.189117
...,...,...,...
2396,Paris,Vienna,1036.040752
2397,Paris,London,343.348168
2398,Paris,Moscow,2486.443932
2399,Paris,Rome,1106.651465


In [23]:
merged_votes = votes_df.merge(countries, left_on="from_country", right_on="code")
merged_votes['from_capital'] = merged_votes["capital"]
merged_votes = merged_votes.drop(["from_country_id", "to_country_id", "name", "code", "capital"], axis=1)
merged_votes = merged_votes.merge(countries, left_on="to_country", right_on="code")
merged_votes['to_capital'] = merged_votes["capital"]
merged_votes = merged_votes.drop(["name", "code", "capital"], axis=1)
merged_votes

,year,round,from_country,to_country,total_points,tele_points,jury_points,from_capital,to_capital
0,1957,final,at,nl,6,NaN,NaN,Vienna,Amsterdam
1,1957,final,at,fr,0,NaN,NaN,Vienna,Paris
2,1957,final,at,dk,0,NaN,NaN,Vienna,København
3,1957,final,at,lu,3,NaN,NaN,Vienna,Luxembourg
4,1957,final,at,de,0,NaN,NaN,Vienna,Berlin
...,...,...,...,...,...,...,...,...,...
51349,2023,final,wld,al,6,6.0,NaN,NaN,Tirana
51350,2023,final,wld,pt,0,0.0,NaN,NaN,Lisbon
51351,2023,final,wld,rs,0,0.0,NaN,NaN,Belgrade
51352,2023,final,wld,gb,0,0.0,NaN,NaN,London


In [28]:
vote_dists = merged_votes.merge(distance_stack, left_on=["from_capital", "to_capital"], right_on=["from", "to"], how="left")
vote_dists = vote_dists.drop(["from", "to"], axis=1)
vote_dists

,year,round,from_country,to_country,total_points,tele_points,jury_points,from_capital,to_capital,distance
0,1957,final,at,nl,6,NaN,NaN,Vienna,Amsterdam,935.926496
1,1957,final,at,fr,0,NaN,NaN,Vienna,Paris,1036.040752
2,1957,final,at,dk,0,NaN,NaN,Vienna,København,871.969165
3,1957,final,at,lu,3,NaN,NaN,Vienna,Luxembourg,765.856559
4,1957,final,at,de,0,NaN,NaN,Vienna,Berlin,524.812724
...,...,...,...,...,...,...,...,...,...,...
51349,2023,final,wld,al,6,6.0,NaN,NaN,Tirana,NaN
51350,2023,final,wld,pt,0,0.0,NaN,NaN,Lisbon,NaN
51351,2023,final,wld,rs,0,0.0,NaN,NaN,Belgrade,NaN
51352,2023,final,wld,gb,0,0.0,NaN,NaN,London,NaN


In [29]:
vote_dists.to_csv("data/vote_distances.csv")